In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
currency_pair = 'USDT_BTC'

In [3]:
folder = '../data/'
files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(folder)) for f in fn]
datafiles = sorted([f for f in files if f.endswith('.log.gz')])
print("Number of datafiles to extract from: {}".format(len(datafiles)))
print("First file: {}".format(datafiles[0]))
print("Last file : {}".format(datafiles[-1]))

Number of datafiles to extract from: 101125
First file: ../data/2016-11-08/poloniex_full_2016-11-08T10:00:09.955811.log.gz
Last file : ../data/2017-01-17/poloniex_full_2017-01-17T16:33:04.583624.log.gz


In [4]:
from manage_orderbooks import *
from orderbook_trade_simulator import *

In [35]:
query_idx = range(0,10)
extract_orderbooks_for_one_currencypair_newSlimDesign([datafiles[idx] for idx in query_idx], currency_pair='USDT_BTC',
                                        outfile='test',
                                        range_factor=1.5, float_precision=2)

 10%|█         | 1/10 [00:00<00:01,  6.59it/s]

Orderbook content will be written to 'test'


100%|██████████| 10/10 [00:01<00:00,  5.92it/s]


-1

In [20]:
filename = '../data/snapshots/orderbooks_USDT_BTC_range1.15_samplesNone_snapshot2017-01-11T17:19.dict'
orderbooks, timestamps = load_orderbook_snapshot(infile=filename, first_line=0, last_line=1000)
# display(timestamps)
print(len(orderbooks[0]))

# orderbooks = orderbooks[:1000]

100%|██████████| 1000/1000 [00:14<00:00, 67.28it/s]

Loaded Orderbooks: 1000
908


In [ ]:
display(timestamps[0][:-10])

In [ ]:
# import warnings
# warnings.filterwarnings('error')

ots = OrderbookTradingSimulator()

vol = 40
period_length = 1
lim = 708.1
order_periods = 10
must_trade = False

ob = orderbooks[0]

for i in range(order_periods):
    if i == order_periods-1:
        must_trade = True
    
    print("Period {}: volume={}, limit={}".format(i+1, vol, lim))
    new_ob = ots.trade_timespan(orderbooks[i*period_length:(i+1)*period_length],
                                timestamps[i*period_length],
                                timespan=period_length, must_trade=must_trade,
                                volume=vol, limit=lim, verbose=False)
    vol = ots.history.volume_left.values[-1]
    
    last_trade = ots.history.volume_traded.values[-1]
    if last_trade != 0:
        print("  Traded: {} shares\n".format(last_trade))
    
    if abs(vol) == 0:
        print("Done after {} order_periods".format(i+1))
        print("Total costs: {}, cotal cashflow: {}".format(sum(ots.history.cost), sum(ots.history.cashflow)))
        break
    elif vol > 0:
        pass
        #lim += 1
    elif vol < 0:
        pass
        #lim -= 1
display(ots.history)
ots.summarize(new_ob)
    

# new_ob = ots.trade_timespan(orderbooks[timespan:timespan+timespan], volume=vol, limit=706, verbose=True, timespan=timespan)
plot_orderbook(new_ob, title = "Orderbook {}".format(ots.t), normalized=True, range_factor=1.05)

In [ ]:
ots = OrderbookTradingSimulator()

vol = 2

ob = orderbooks[0]
new_ob = ots.trade(orderbooks[0], volume=vol, limit=None, verbose=True)
plot_orderbook(new_ob, title = "Orderbook {}".format(ots.t), normalized=True, range_factor=1.05)
trade_counter = 1

In [ ]:
center = orderbooks[0][orderbooks[0].Type=='center'].Price.values[0]
bid = orderbooks[0][orderbooks[0].Type=='bid'].Price.values[-1]
ask = orderbooks[0][orderbooks[0].Type=='ask'].Price.values[0]
print("Bid:    {}".format(bid))
print("Center: {}".format(center))
print("Ask:    {}\n".format(ask))

result = 0
trade_vol = -10
limit=704
trade_info = place_order(orderbooks[0], trade_vol, limit=limit)
cash_flow = trade_info['cashflow']
trade_vol -= trade_info['amount_fulfilled']

display(trade_info)
display(orderbook_preview(ob, 3))

#cash_flow = 0
minute = 1
for i, ob in enumerate(orderbooks[1:]):
    if abs(trade_vol) < 1.0e-10:
        break
        
    
    print("timesteps: {} ==> {}, {}".format(minute, cash_flow, trade_vol))
    if abs(trade_vol) > 1.0e-10 and i == len(orderbooks[1:])-1:
        print("TRADE!!!")
        # trade always in last Timestep
        trade_info = place_order(ob,  trade_vol, limit=None, trade_history=trade_info['trade_history'])
        cash_flow += trade_info['cashflow']
        trade_vol -= trade_info['amount_fulfilled']
    
    elif abs(trade_vol) > 1.0e-10:
        print("trade_vol", trade_vol)
        trade_info = place_order(ob,  trade_vol, limit=limit, trade_history=trade_info['trade_history'])
        cash_flow += trade_info['cashflow']
        trade_vol -= trade_info['amount_fulfilled']

    display(trade_info)
    
    minute += 1
    
    print("")
    
    print("trade_vol: {:1.4f}, center: {:1.4f}, limit: {}".format(trade_vol, ob[ob.Type=='center'].Price.values[0], limit))
print("timesteps: {} ==> {}, {}".format(minute, cash_flow, trade_vol))

display(orderbook_preview(ob, 3))

In [ ]:
range_factor = 1.01
# plot_orderbook(data[0], normalized=True, range_factor=range_factor)
print(len(orderbooks))
for idx in [1, 5000, 25000, -1]:
    print(idx, timestamps[idx])
    display(orderbook_preview(orderbooks[idx], 3))
    plot_orderbook(orderbooks[idx], title=timestamps[idx], normalized=True, range_factor=range_factor)